In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.transforms as transforms
# from torchvision.datasets import FashionMNIST
# from torch.utils.data import DataLoader
# import torch.nn.functional as F
# import pandas as pd
# from tabulate import tabulate

# # Hyperparameters
# batch_size = 64
# learning_rate = 0.001
# num_epochs = 300
# confidence_margin = 0.2
# unconfident_penalty = 5.0
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Transformations with data augmentation and normalization
# transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ColorJitter(brightness=0.1, contrast=0.1),
#     transforms.RandomCrop(28, padding=4),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))
# ])

# # Load FashionMNIST dataset
# train_dataset = FashionMNIST(root="./data", train=True, transform=transform, download=True)
# test_dataset = FashionMNIST(root="./data", train=False, transform=transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))
# ]), download=True)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# actual_labels = [label for _, labels in test_loader for label in labels.numpy()]

# # Define CNN model with classwise confidence thresholds as learnable parameters
# class HighConfidenceFashionCNN(nn.Module):
#     def __init__(self, num_classes=10, init_threshold=0.9):
#         super(HighConfidenceFashionCNN, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(32)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.bn2 = nn.BatchNorm2d(64)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.bn3 = nn.BatchNorm2d(128)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.fc1 = nn.Linear(128 * 3 * 3, 256)
#         self.dropout1 = nn.Dropout(0.4)
#         self.fc2 = nn.Linear(256, 128)
#         self.dropout2 = nn.Dropout(0.4)
#         self.fc3 = nn.Linear(128, num_classes)
        
#         # Initialize classwise confidence thresholds as learnable parameters
#         self.classwise_thresholds = nn.Parameter(init_threshold * torch.ones(num_classes))

#     def forward(self, x):
#         x = self.pool(F.relu(self.bn1(self.conv1(x))))
#         x = self.pool(F.relu(self.bn2(self.conv2(x))))
#         x = self.pool(F.relu(self.bn3(self.conv3(x))))
#         x = x.view(-1, 128 * 3 * 3)
#         x = F.relu(self.fc1(x))
#         x = self.dropout1(x)
#         x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = self.fc3(x)
#         return x

# model = HighConfidenceFashionCNN().to(device)

# # Optimizer and scheduler
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=num_epochs, anneal_strategy="linear"
# )

# # Improved loss function to include classwise thresholds
# # Improved custom loss function aiming for smoother, convex-like behavior
# def custom_loss(outputs, labels, model, confidence_margin=0.2, penalty_weight=unconfident_penalty):
#     # Standard cross-entropy loss
#     base_loss = F.cross_entropy(outputs, labels, reduction='none')
    
#     # Apply softmax to get probabilities
#     probs = F.softmax(outputs, dim=1)
#     top2_probs, _ = torch.topk(probs, 2, dim=1)
#     top_class_prob = top2_probs[:, 0]
#     second_class_prob = top2_probs[:, 1]
    
#     # Compute class-specific thresholds from model's learnable parameters
#     class_thresholds = model.classwise_thresholds[labels]

#     # Quadratic penalty based on confidence deviation from margin
#     confidence_margin_penalty = penalty_weight * ((top_class_prob - second_class_prob - confidence_margin) ** 2)
    
#     # Threshold penalty as a quadratic deviation from classwise thresholds
#     threshold_regularization = penalty_weight * ((top_class_prob - class_thresholds) ** 2)

#     # Combine base loss, confidence margin penalty, and threshold regularization
#     loss = base_loss + confidence_margin_penalty + threshold_regularization
#     return loss.mean()


# # Training function
# def train(model, loader, optimizer, epoch, max_epochs):
#     model.train()
#     total_loss = 0.0
    
#     for images, labels in loader:
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = custom_loss(outputs, labels, model)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     return total_loss / len(loader)

# # Evaluation function that uses classwise thresholds
# def evaluate(model, loader):
#     model.eval()
#     sample_ids, predicted_labels = [], []

#     with torch.no_grad():
#         for batch_idx, (images, labels) in enumerate(loader):
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             probs = F.softmax(outputs, dim=1)
            
#             # Get top 2 predictions for each sample
#             top2_probs, top2_classes = torch.topk(probs, 2, dim=1)

#             for i in range(len(labels)):
#                 sample_id = batch_idx * batch_size + i
#                 top_class_prob = top2_probs[i][0].item()
#                 second_class_prob = top2_probs[i][1].item()
#                 top_class = top2_classes[i][0].item()
                
#                 # Class-specific threshold from the model
#                 class_threshold = model.classwise_thresholds[top_class].item()
                
#                 # Prediction decision based on classwise threshold
#                 if (top_class_prob - second_class_prob) >= confidence_margin and top_class_prob >= class_threshold:
#                     predicted_class = top_class
#                 else:
#                     predicted_class = -1

#                 sample_ids.append(sample_id)
#                 predicted_labels.append(predicted_class)

#     return sample_ids, predicted_labels

# def calculate_summary(actual_labels, predicted_labels, confidence_threshold=0.90):
#     # Determine the number of classes dynamically
#     classes = sorted(set(actual_labels) - {-1})
#     num_classes = len(classes)

#     # Initialize summary data
#     class_counts = {cls: 0 for cls in classes}
#     predictions_made = {cls: 0 for cls in classes}
#     correct_predictions = {cls: 0 for cls in classes}
#     unconfident_predictions = {cls: 0 for cls in classes}
    
#     # Additional thresholds for score calculation
#     thresholds = [0.90, 0.95, 0.99]
#     scores_for_thresholds = {f"Score_{threshold}": {cls: 0 for cls in classes} for threshold in thresholds}
#     scores_for_thresholds["Score_Contribution"] = {cls: 0 for cls in classes}  # Custom threshold score

#     # Process each prediction
#     for actual, predicted in zip(actual_labels, predicted_labels):
#         class_counts[actual] += 1
#         if predicted == actual:
#             correct_predictions[actual] += 1
#         if predicted == -1:
#             unconfident_predictions[actual] += 1
#         elif predicted in classes:
#             predictions_made[predicted] += 1
    
#     # Calculate accuracy and score contribution for each class and each threshold
#     data = []
#     for cls in classes:
#         if predictions_made[cls] > 0:
#             accuracy = correct_predictions[cls] / predictions_made[cls]
#         else:
#             accuracy = 0

#         # Calculate score contributions for each threshold
#         score_contributions = {}
#         for threshold in thresholds:
#             if accuracy >= threshold:
#                 score_contribution = correct_predictions[cls]
#             else:
#                 score_contribution = -2 * predictions_made[cls]
#             score_contributions[f"Score_{threshold}"] = score_contribution

#         # Calculate score contribution for the custom threshold
#         if accuracy >= confidence_threshold:
#             score_contribution = correct_predictions[cls]
#         else:
#             score_contribution = -2 * predictions_made[cls]

#         # Append the row to data with all calculated values
#         data.append({
#             "Class": cls,
#             "Actual": class_counts[cls],
#             "Predicted": predictions_made[cls],
#             "Correct": correct_predictions[cls],
#             "Incorrect": predictions_made[cls] - correct_predictions[cls],
#             "Unconfident (-1)": unconfident_predictions[cls],
#             "Accuracy": accuracy,
#             "Score_Contribution": score_contribution,  # Custom threshold column
#             **score_contributions  # Add other threshold score columns dynamically
#         })

#     # Create a summary DataFrame with scores for each threshold
#     summary_df = pd.DataFrame(data)

#     # Calculate totals for each relevant column
#     total_row = {
#         "Class": "Total",
#         "Actual": sum(class_counts.values()),
#         "Predicted": sum(predictions_made.values()),
#         "Correct": sum(correct_predictions.values()),
#         "Incorrect": sum(predictions_made.values()) - sum(correct_predictions.values()),
#         "Unconfident (-1)": sum(unconfident_predictions.values()),
#         "Accuracy": "",  # Leave Accuracy empty in the total row
#         "Score_Contribution": summary_df["Score_Contribution"].sum(),  # Custom threshold total
#     }

#     # Add totals for each additional score threshold
#     for threshold in thresholds:
#         total_row[f"Score_{threshold}"] = summary_df[f"Score_{threshold}"].sum()

#     # Convert total_row to DataFrame and use pd.concat to add it to summary_df
#     total_df = pd.DataFrame([total_row])
#     summary_df = pd.concat([summary_df, total_df], ignore_index=True)

#     return summary_df
    
# # Call training and evaluation with logging at specific intervals
# for epoch in range(num_epochs):
#     avg_train_loss = train(model, train_loader, optimizer, epoch, num_epochs)
#     print(f"Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {avg_train_loss:.4f}")
#     scheduler.step()

#     if epoch % 10 == 0 or epoch >= num_epochs - 10:
#         sample_ids, predicted_labels = evaluate(model, test_loader)
#         summary_df = calculate_summary(actual_labels, predicted_labels, confidence_threshold=0.99)
#         print(f"\nSummary for Confidence Threshold 0.99 - Epoch {epoch+1}")
#         print(tabulate(summary_df, headers='keys', tablefmt='fancy_grid'))


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import pandas as pd
from tabulate import tabulate

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 300
confidence_margin = 0.2
unconfident_penalty = 5.0
penalty_weight = 0.01  # Weight for the validation-based penalty
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations with data augmentation and normalization
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.RandomCrop(28, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load FashionMNIST dataset and split into training and validation sets
train_dataset = FashionMNIST(root="./data", train=True, transform=transform, download=True)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Load test dataset
test_dataset = FashionMNIST(root="./data", train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
]), download=True)

# Data loaders
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

actual_labels = [label for _, labels in test_loader for label in labels.numpy()]

# Define CNN model with classwise confidence thresholds as learnable parameters
class HighConfidenceFashionCNN(nn.Module):
    def __init__(self, num_classes=10, init_threshold=0.9):
        super(HighConfidenceFashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(128, num_classes)
        
        # Initialize classwise confidence thresholds as learnable parameters
        self.classwise_thresholds = nn.Parameter(init_threshold * torch.ones(num_classes))

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

model = HighConfidenceFashionCNN().to(device)

# Optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=num_epochs, anneal_strategy="linear"
)

# Improved custom loss function with validation loss regularization
def custom_loss(outputs, labels, model, val_loss, confidence_margin=0.2, penalty_weight=unconfident_penalty):
    base_loss = F.cross_entropy(outputs, labels, reduction='none')
    
    # Apply softmax to get probabilities
    probs = F.softmax(outputs, dim=1)
    top2_probs, _ = torch.topk(probs, 2, dim=1)
    top_class_prob = top2_probs[:, 0]
    second_class_prob = top2_probs[:, 1]
    
    # Compute class-specific thresholds from model's learnable parameters
    class_thresholds = model.classwise_thresholds[labels]

    # Quadratic penalty based on confidence deviation from margin
    confidence_margin_penalty = penalty_weight * ((top_class_prob - second_class_prob - confidence_margin) ** 2)
    
    # Threshold penalty as a quadratic deviation from classwise thresholds
    threshold_regularization = penalty_weight * ((top_class_prob - class_thresholds) ** 2)

    # Add validation loss regularization
    regularized_loss = base_loss + confidence_margin_penalty + threshold_regularization + penalty_weight * val_loss
    return regularized_loss.mean()

# Training function
def train(model, loader, optimizer, epoch, max_epochs, val_loss):
    model.train()
    total_loss = 0.0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = custom_loss(outputs, labels, model, val_loss)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# Validation function
def validate(model, loader):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)  # Regular cross-entropy loss for validation
            val_loss += loss.item()
    return val_loss / len(loader)

# Evaluation function that uses classwise thresholds
def evaluate(model, loader):
    model.eval()
    sample_ids, predicted_labels = [], []

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)
            
            # Get top 2 predictions for each sample
            top2_probs, top2_classes = torch.topk(probs, 2, dim=1)

            for i in range(len(labels)):
                sample_id = batch_idx * batch_size + i
                top_class_prob = top2_probs[i][0].item()
                second_class_prob = top2_probs[i][1].item()
                top_class = top2_classes[i][0].item()
                
                # Class-specific threshold from the model
                class_threshold = model.classwise_thresholds[top_class].item()
                
                # Prediction decision based on classwise threshold
                if (top_class_prob - second_class_prob) >= confidence_margin and top_class_prob >= class_threshold:
                    predicted_class = top_class
                else:
                    predicted_class = -1

                sample_ids.append(sample_id)
                predicted_labels.append(predicted_class)

    return sample_ids, predicted_labels


def calculate_summary(actual_labels, predicted_labels, confidence_threshold=0.90):
    # Determine the number of classes dynamically
    classes = sorted(set(actual_labels) - {-1})
    num_classes = len(classes)

    # Initialize summary data
    class_counts = {cls: 0 for cls in classes}
    predictions_made = {cls: 0 for cls in classes}
    correct_predictions = {cls: 0 for cls in classes}
    unconfident_predictions = {cls: 0 for cls in classes}
    
    # Additional thresholds for score calculation
    thresholds = [0.90, 0.95, 0.99]
    scores_for_thresholds = {f"Score_{threshold}": {cls: 0 for cls in classes} for threshold in thresholds}
    scores_for_thresholds["Score_Contribution"] = {cls: 0 for cls in classes}  # Custom threshold score

    # Process each prediction
    for actual, predicted in zip(actual_labels, predicted_labels):
        class_counts[actual] += 1
        if predicted == actual:
            correct_predictions[actual] += 1
        if predicted == -1:
            unconfident_predictions[actual] += 1
        elif predicted in classes:
            predictions_made[predicted] += 1
    
    # Calculate accuracy and score contribution for each class and each threshold
    data = []
    for cls in classes:
        if predictions_made[cls] > 0:
            accuracy = correct_predictions[cls] / predictions_made[cls]
        else:
            accuracy = 0

        # Calculate score contributions for each threshold
        score_contributions = {}
        for threshold in thresholds:
            if accuracy >= threshold:
                score_contribution = correct_predictions[cls]
            else:
                score_contribution = -2 * predictions_made[cls]
            score_contributions[f"Score_{threshold}"] = score_contribution

        # Calculate score contribution for the custom threshold
        if accuracy >= confidence_threshold:
            score_contribution = correct_predictions[cls]
        else:
            score_contribution = -2 * predictions_made[cls]

        # Append the row to data with all calculated values
        data.append({
            "Class": cls,
            "Actual": class_counts[cls],
            "Predicted": predictions_made[cls],
            "Correct": correct_predictions[cls],
            "Incorrect": predictions_made[cls] - correct_predictions[cls],
            "Unconfident (-1)": unconfident_predictions[cls],
            "Accuracy": accuracy,
            "Score_Contribution": score_contribution,  # Custom threshold column
            **score_contributions  # Add other threshold score columns dynamically
        })

    # Create a summary DataFrame with scores for each threshold
    summary_df = pd.DataFrame(data)

    # Calculate totals for each relevant column
    total_row = {
        "Class": "Total",
        "Actual": sum(class_counts.values()),
        "Predicted": sum(predictions_made.values()),
        "Correct": sum(correct_predictions.values()),
        "Incorrect": sum(predictions_made.values()) - sum(correct_predictions.values()),
        "Unconfident (-1)": sum(unconfident_predictions.values()),
        "Accuracy": "",  # Leave Accuracy empty in the total row
        "Score_Contribution": summary_df["Score_Contribution"].sum(),  # Custom threshold total
    }

    # Add totals for each additional score threshold
    for threshold in thresholds:
        total_row[f"Score_{threshold}"] = summary_df[f"Score_{threshold}"].sum()

    # Convert total_row to DataFrame and use pd.concat to add it to summary_df
    total_df = pd.DataFrame([total_row])
    summary_df = pd.concat([summary_df, total_df], ignore_index=True)

    return summary_df
    
# Calculate validation loss
for epoch in range(num_epochs):
    val_loss = validate(model, val_loader)  # Calculate validation loss
    avg_train_loss = train(model, train_loader, optimizer, epoch, num_epochs, val_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")
    scheduler.step()

    # Evaluate and print summary only after certain epochs
    if epoch % 10 == 0 or epoch >= num_epochs - 10:
        sample_ids, predicted_labels = evaluate(model, test_loader)
        summary_df = calculate_summary(actual_labels, predicted_labels, confidence_threshold=0.99)
        print(f"\nSummary for Confidence Threshold 0.99 - Epoch {epoch+1}")
        print(tabulate(summary_df, headers='keys', tablefmt='fancy_grid'))


100%|██████████| 26421880/26421880 [00:02<00:00, 12557110.45it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 207106.29it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3880931.70it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11766908.44it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch [1/300], Train Loss: 13.4725, Val Loss: 2.3037

Summary for Confidence Threshold 0.99 - Epoch 1
╒════╤═════════╤══════════╤═════════════╤═══════════╤═════════════╤════════════════════╤════════════════════╤══════════════════════╤═════════════╤══════════════╤══════════════╕
│    │ Class   │   Actual │   Predicted │   Correct │   Incorrect │   Unconfident (-1) │ Accuracy           │   Score_Contribution │   Score_0.9 │   Score_0.95 │   Score_0.99 │
╞════╪═════════╪══════════╪═════════════╪═══════════╪═════════════╪════════════════════╪════════════════════╪══════════════════════╪═════════════╪══════════════╪══════════════╡
│  0 │ 0       │     1000 │           0 │         0 │           0 │               1000 │ 0.0                │                    0 │           0 │            0 │            0 │
├────┼─────────┼──────────┼─────────────┼───────────┼─────────────┼────────────────────┼─────────────